In [ ]:
# Change working directory to Colab Notebooks.
%cd "/content/drive/My Drive/Colab/"

In [ ]:
cloud_storage = False
test = False

if cloud_storage:
    
    from google.colab import auth
    auth.authenticate_user()
    project_id = 'intphys'
    bucket_name = 'datasets-intphys'
    !gcloud config set project {project_id}
    !gsutil cp gs://{bucket_name}/dataset-intphys-15000.hdf5 /content/temp_dataset/dataset-intphys-15000.hdf5

elif test:
    !mkdir /content/temp_dataset
    !cp dataset/dataset-intphys-test.hdf5 /content/temp_dataset


In [ ]:
# Install missing packages.

# System libraries.
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"  # Avoid most Tensorflow warning errors.
import sys

# Maths and utilites.
import numpy as np
import matplotlib.pyplot as plt
import h5py

# Keras and Tensorflow.
import tensorflow as tf
%tensorflow_version 
from tensorflow.keras.models import load_model, save_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger
from tensorflow.keras.utils import plot_model

# Import my modules.
sys.path.append("./modules")
from partition import data_dictionaries_depth
from generators import DepthDataGenerator
from model_unet import get_unet_depth
from losses_prediction import intensity
from visualization import prediction_mask_object, prediction_mask_occlu, unet_seg

In [ ]:
# Check if GPU is being used.
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
    print('GPU device not found')
    USE_GPU = False
else:
    print('Found GPU at: {}'.format(device_name))
    # !nvidia-smi  # GPU details.
    USE_GPU = True

In [ ]:
# Filename.
filename_unet = "unet-depth-L1-08"
# Create Unet model.
unet_model = get_unet_depth()
# unet_model.summary()

In [ ]:
# Losses and metrics. 
intensity_loss = intensity(l_num=1)

In [ ]:
# Compile model
unet_model.compile(loss="mae", optimizer=Adam(learning_rate=1e-3), metrics=["mae", "mse"])

In [ ]:
# Create partition dictionaries
path_hdf5 = "/content/temp_dataset/dataset-intphys-08000.hdf5"
partition, targets = data_dictionaries_depth(num_initial=7001, num_scenes=1000, path_hdf5=path_hdf5)

# Parameters.
params = {'dim': (288, 288),
          'path_hdf5': path_hdf5,
          'scale' : 0.5,
          'batch_size': 32,
          'num_channels': 3,
          'shuffle': True}

# Generators.
training_generator = DepthDataGenerator(partition["train"], targets, **params)
validation_generator = DepthDataGenerator(partition["validation"], targets, **params)

In [ ]:
# Select model for training.
model = unet_model
filename = filename_unet
model_clean_weights = model.get_weights()

In [ ]:
# Configure Keras callbacks for training.
model_checkpoint = ModelCheckpoint(filepath="./models/{}.h5".format(filename),
                                   save_best_only=False,
                                   verbose=1)

early_stopping = EarlyStopping(patience=4,
                               restore_best_weights=False,
                               verbose=1)

reduce_lr = ReduceLROnPlateau(factor=0.1,
                              patience=2,
                              min_lr=1e-6, 
                              verbose=1)

csv_log = CSVLogger(filename="./logs/{}.csv".format(filename),
                    separator=";")

cb_list = [model_checkpoint, csv_log]

In [ ]:
# Load precious model.
model_loading = True
if model_loading:
    model = load_model("./models/unet-depth-L1-07.h5")

In [ ]:
# Clean weights before training
if not model_loading:
    model.set_weights(model_clean_weights)

# Generator training.
train_history = model.fit(x=training_generator,
                          validation_data=validation_generator,
                          callbacks=cb_list,
                          epochs=5)

In [ ]:
prediction_depth(path_hdf5, model, scale=0.5, scene=7960)

In [ ]:
model = load_model("models/unet-depth-L1-good.h5")

In [ ]:
block_number = 12
print("model-{:02}000".format(block_number))